<a href="https://colab.research.google.com/github/Hummna/Core-Human-Facial-Emotions-Detection./blob/main/Models_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:


import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

from tqdm.notebook import tqdm_notebook as tqdm

import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data Preprocessing

In [ ]:


print(os.listdir("/content/drive/MyDrive/Colab Notebooks/train"))

In [ ]:



angry= '/content/drive/MyDrive/Colab Notebooks/train/angry'
disgust = '/content/drive/MyDrive/Colab Notebooks/train/disgust'
fear = '/content/drive/MyDrive/Colab Notebooks/train/fear'
neutral = '/content/drive/MyDrive/Colab Notebooks/train/neutral'
surprise = '/content/drive/MyDrive/Colab Notebooks/train/surprise'




angry_path = os.listdir(angry)
disgust_path = os.listdir(disgust)
fear_path = os.listdir(fear)
neutral_path = os.listdir(neutral)
surprise_path = os.listdir(surprise)

# Sample Images

In [ ]:

def load_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image,(224, 224))
    return image[...,::-1]

In [ ]:


def load_img(path):
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded successfully
        image = cv2.resize(image, (224, 224))
        return image[...,::-1]
    else:
        print(f"/content/drive/MyDrive/Colab Notebooks/train/angry: {path}")
        return None


In [ ]:
def load_img(path):
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded successfully
        image = cv2.resize(image, (224, 224))
        return image[...,::-1]
    else:
        print(f"/content/drive/MyDrive/Colab Notebooks/train/disgust: {path}")
        return None


In [ ]:
def load_img(path):
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded successfully
        image = cv2.resize(image, (224, 224))
        return image[...,::-1]
    else:
        print(f"/content/drive/MyDrive/Colab Notebooks/train/fear: {path}")
        return None

In [ ]:
def load_img(path):
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded successfully
        image = cv2.resize(image, (224, 224))
        return image[...,::-1]
    else:
        print(f"/content/drive/MyDrive/Colab Notebooks/train/neutral: {path}")
        return None

In [ ]:
def load_img(path):
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded successfully
        image = cv2.resize(image, (224, 224))
        return image[...,::-1]
    else:
        print(f"/content/drive/MyDrive/Colab Notebooks/train/surprise: {path}")
        return None

# Modeling

In [ ]:


dataset_path = "/content/drive/MyDrive/Colab Notebooks/train"

In [ ]:


data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.2)

In [ ]:


train = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="training")

In [ ]:


val = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="validation"
                                          )

# 1) MobileNetV2

In [ ]:


mnet = MobileNetV2(include_top = False, weights = "imagenet" ,input_shape=(96,96,3))

In [ ]:


tf.keras.backend.clear_session()

model = Sequential([mnet,
                    GlobalAveragePooling2D(),
                    Dense(512, activation = "relu"),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(128, activation = "relu"),
                    Dropout(0.1),
                    # Dense(32, activation = "relu"),
                    # Dropout(0.3),
                    Dense(5, activation = "sigmoid")])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

model.summary()
Model: "sequential"

In [ ]:

def scheduler(epoch):
    if epoch <= 1:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:


hist = model.fit_generator(train,
                    epochs=5,
                    callbacks=[lr_callbacks],
                    validation_data=val)

In [ ]:


epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

# 2) VGG16

In [ ]:


train = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=98,
                                          subset="training")

val = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=98,
                                          subset="validation"
                                          )

In [ ]:


vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))

In [ ]:


vgg16_model.output[-1]

In [ ]:


# model = Sequential()
# for layer in vgg16_model.layers[:-1]:
#     model.add(layer)

# for layer in model.layers:
#     layer.trainable = False

# model.add(Dense(2, activation='softmax'))


model = Sequential([vgg16_model,
                    Flatten(),
#                     GlobalAveragePooling2D(),
#                     Dense(512, activation = "relu"),
#                     BatchNormalization(),
#                     Dropout(0.3),
#                     Dense(128, activation = "relu"),
#                     Dropout(0.1),
#                     # Dense(32, activation = "relu"),
#                     # Dropout(0.3),
                    Dense(5, activation = "sigmoid")])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

model.summary()

In [ ]:


hist =  model.fit_generator(train,
                    epochs=5,
                    callbacks=[lr_callbacks],
                    validation_data=val)

<ipython-input-49-ea3365b6907f>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist =  model.fit_generator(train,


Epoch 1/5
77/77 [==============================] - 5202s 68s/step - loss: 0.8509 - accuracy: 0.6654 - val_loss: 0.6609 - val_accuracy: 0.7496 - lr: 0.0010
Epoch 2/5
 3/77 [>.............................] - ETA: 1:06:11 - loss: 0.5594 - accuracy: 0.8231

In [ ]:


epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

### 3)**InceptionV3**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the InceptionV3 model pre-trained on ImageNet data
inception_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    inception_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(5, activation="sigmoid")
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

# **4)DenseNet201**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the DenseNet201 model pre-trained on ImageNet data
densenet201_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    densenet201_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(5, activation="softmax")  # Adjust the number of units for your specific problem (multi-class classification)
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 05
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

# 5)**DenseNet169**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the DenseNet169 model pre-trained on ImageNet data
densenet169_model = DenseNet169(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    densenet169_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(5, activation="softmax")  # Adjust the number of units for your specific problem (multi-class classification)
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])



# 6)**VGG19**



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the VGG19 model pre-trained on ImageNet data
vgg19_model = VGG19(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    vgg19_model,
    Flatten(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(5, activation="softmax")  # Adjust the number of units for your specific problem (multi-class classification)
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

# 7)**ResNet152**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the ResNet152 model pre-trained on ImageNet data
resnet152_model = ResNet152(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    resnet152_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(4, activation="sigmoid")
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

# 8)**ResNet101**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load the ResNet101 model pre-trained on ImageNet data
resnet101_model = ResNet101(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

tf.keras.backend.clear_session()

model = Sequential([
    resnet101_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(5, activation="sigmoid")
])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Define a learning rate scheduler function
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif 2 < epoch <= 15:
        return 0.0001
    else:
        return 0.00001

# Create a LearningRateScheduler callback
lr_callbacks = LearningRateScheduler(scheduler)

# Assuming you have already defined train and val data generators
hist = model.fit(train, epochs=5, callbacks=[lr_callbacks], validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])